In [ ]:
! pip install transformers

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
device = "cuda" if torch. cuda. is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [3]:
input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt" )["input_ids" ].to(device)
output = model.generate(input_ids, max_new_tokens= 8 , do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world,


In [7]:

# first we prepare the our input by tokenize it and specify number of steps we want
# and how many choices we get at each step
input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt" )["input_ids" ] . to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
  for _ in range(n_steps):
      # first time we will decode the input and then every time we will decode the input with last output to show them latter at each step
      iteration = dict()
      iteration["Input" ] = tokenizer.decode(input_ids[0])
      # then pa
      output = model(input_ids=input_ids)

      # We will Select logits of the first batch and the last token and apply softmax
      next_token_logits = output.logits[0, - 1, :]
      next_token_probs = torch.softmax(next_token_logits, dim=- 1)
      # we will sort the tokens probability
      sorted_ids = torch.argsort(next_token_probs, dim=- 1, descending=True)

      #  then Store tokens with highest probabilities at each step
      for choice_idx in range(choices_per_step):
          token_id = sorted_ids[choice_idx]
          token_prob = next_token_probs[token_id] .cpu().numpy()
          token_choice = (
          f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
          )
          iteration[f"Choice {choice_idx+1}" ] = token_choice

      # concatenate the predicted next token to the input
      input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=- 1)
      iterations.append(iteration )


# now we can see how the output is at each step
import pandas as pd
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (8.53%),only (4.96%),best (4.65%),Transformers (4.37%),ultimate (2.16%)
1,Transformers are the most,popular (16.78%),powerful (5.37%),common (4.96%),famous (3.72%),successful (3.20%)
2,Transformers are the most popular,toy (10.63%),toys (7.23%),Transformers (6.60%),of (5.46%),and (3.76%)
3,Transformers are the most popular toy,line (34.38%),in (18.20%),of (11.71%),brand (6.10%),line (2.69%)
4,Transformers are the most popular toy line,in (46.28%),of (15.09%),", (4.94%)",on (4.40%),ever (2.72%)
5,Transformers are the most popular toy line in,the (65.99%),history (12.42%),America (6.91%),Japan (2.44%),North (1.40%)
6,Transformers are the most popular toy line in the,world (69.26%),United (4.55%),history (4.29%),US (4.23%),U (2.30%)
7,Transformers are the most popular toy line in ...,", (39.73%)",. (30.64%),and (9.87%),with (2.32%),today (1.74%)


In [14]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English. \n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt" )["input_ids" ].to(device)
output_greedy = model.generate(input_ids, max_length=max_length,
do_sample=False)
print(tokenizer. decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees. 


The researchers were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees. 


The researchers were conducting a study on
